In [1]:
!pip install transformers datasets scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 2

In [8]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer
import numpy as np
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments, pipeline
from datasets import load_metric

In [10]:
# dataset
data = pd.read_csv('/content/greetings_dataset.csv')
dataset = Dataset.from_pandas(data)

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Tokenizar
def tokenize_function(examples):
    return tokenizer(examples['texto'], padding='max_length', truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

labels_to_ids = {'greeting': 0, 'none': 1}
tokenized_datasets = tokenized_datasets.map(lambda x: {'label': labels_to_ids[x['label']]})

id2label = {0: 'greeting', 1: 'not_greeting'}
label2id = {v: k for k, v in id2label.items()}

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [11]:
# BERT
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
model.config.id2label = id2label
model.config.label2id = label2id

tokenized_datasets = tokenized_datasets.train_test_split(test_size=0.2)

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics,
)

# 🦾🦾
trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.036895,1.000000
2,No log,0.003338,1.000000
3,No log,0.002393,1.000000


TrainOutput(global_step=150, training_loss=0.10691238403320312, metrics={'train_runtime': 55.9677, 'train_samples_per_second': 42.882, 'train_steps_per_second': 2.68, 'total_flos': 631466532864000.0, 'train_loss': 0.10691238403320312, 'epoch': 3.0})

In [12]:
results = trainer.evaluate()
print(f"Accuracy: {results['eval_accuracy']}")

Accuracy: 1.0


In [14]:
model.save_pretrained("HelloBERT")
tokenizer.save_pretrained("HelloBERT")

('HelloBERT/tokenizer_config.json',
 'HelloBERT/special_tokens_map.json',
 'HelloBERT/vocab.txt',
 'HelloBERT/added_tokens.json',
 'HelloBERT/tokenizer.json')

In [16]:
model = AutoModelForSequenceClassification.from_pretrained("HelloBERT")
tokenizer = AutoTokenizer.from_pretrained("HelloBERT")

# Crear el clasificador usando el modelo cargado y la GPU
classifier = pipeline('text-classification', model=model, tokenizer=tokenizer, device=0)

# Textos para clasificación
texts = [
    "Hello, how are you?",
    "Hola, ¿cómo estás?",
    "What's the weather like today?",
    "Buenos días, ¿cómo te encuentras?",
    "I need help with my homework.",
    "Qué haces panita? Cómo va?",
    "Buenardo, buen día, coscu",
    "heyyyy",
    "buenas buenas",
    "helloooooo",
    "hi",
    "whats uuuuppp????"
]

# Clasificar cada texto y obtener etiquetas directamente
for text in texts:
    result = classifier(text)
    label = result[0]['label']
    print(f"Text: {text} => Label: {label}, Score: {result[0]['score']}")


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Text: Hello, how are you? => Label: greeting, Score: 0.9980661273002625
Text: Hola, ¿cómo estás? => Label: greeting, Score: 0.9977893829345703
Text: What's the weather like today? => Label: not_greeting, Score: 0.9933899641036987
Text: Buenos días, ¿cómo te encuentras? => Label: greeting, Score: 0.99715256690979
Text: I need help with my homework. => Label: not_greeting, Score: 0.9958699345588684
Text: Qué haces panita? Cómo va? => Label: greeting, Score: 0.9890562295913696
Text: Buenardo, buen día, coscu => Label: greeting, Score: 0.9166218638420105
Text: heyyyy => Label: greeting, Score: 0.9971520900726318
Text: buenas buenas => Label: greeting, Score: 0.997988224029541
Text: helloooooo => Label: greeting, Score: 0.9977092742919922
Text: hi => Label: greeting, Score: 0.9982302784919739
Text: whats uuuuppp???? => Label: greeting, Score: 0.9894077181816101
